### RAG 기반 고객사 자체 문서에 대한 커스텀 검색 엔진 제작

In [1]:
%pip install llama-index
%pip install openai
%pip install docx2txt
%pip install torch transformers python-pptx Pillow
%pip install chromadb
%pip install llama-index-retrievers-bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 16.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.1/273.1 kB 34.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=89ef1958c6918ed46d4c6b154abdb24ba133d769cf2e2388829399913cf75bbe
  Stored in directory: /Users/jikime/Library/Caches/pip/wheels/6f/81/48/001bbc0109c15e18c009eee300022f42d1e070e54f1d00b218
Successfully built docx2txt
  Using cached python_pptx-0.6.23-py3-none-any.whl.metadata (18 kB)
  Using cached XlsxWriter-3.2.0-py3-none-any.whl.metadata (2.6 kB)
Using cached python_pptx-0.6.23-py3-none-any.whl (471 kB)
Using cached XlsxWriter-3.2.0-py3-none-any.whl (159 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### llama-index 사용

1. LLM 사용하기

In [3]:
from llama_index.llms.openai import OpenAI

response = OpenAI().complete("삼성전자는 어떤 회사인가요?")
print(response)

삼성전자는 대한민국의 다국적 전자 제품 제조 기업으로, 스마트폰, TV, 가전제품, 반도체 등 다양한 제품을 생산하고 있습니다. 세계적으로 가장 큰 전자 기업 중 하나로, 기술 혁신과 품질에 대한 높은 평가를 받고 있습니다.


In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

## settings 에 llm을 등록하여 llama index의 다른 기능들에서 llm을 사용할 수 있다
Settings.llm = OpenAI(temperature=0.3, model="gpt-4o-mini")

folder_path = '../Assets/data/docx'
documents = SimpleDirectoryReader(folder_path).load_data()
index = VectorStoreIndex.from_documents(
    documents,
)

2. Retreiver 사용해보기

In [4]:
vector_retriever = index.as_retriever(similarity_top_k=2)


In [8]:
docs = await vector_retriever.aretrieve("사우회는 어떻게 운영되나요?")
for doc in docs:
  print(f"Score: {doc.get_score()}")
  print(doc.get_content())
  print("\n\n")


Score: 0.8237742728091122
사 우 회 규 정

제 1 장 총  칙(제1조～제4조)

제 1 조【명칭】
  이 회는 주식회사 사우회(이하 "이 회"라 한다)로 부른다.

제 2 조【목적】
  이 회는 회원의 상호부조와 복지향상을 도모함을 목적으로 한다.

제 3 조【사업】
  이 회는 전조의 목적을 달성하기 위하여 다음의 사항을 행한다.
    1. 상조사업
    2. 기타 이 회의 목적을 달성함에 필요한 사업

제 4 조【기금】
  이 회의 기금은 회비, 기부금 또는 찬조금 및 이 회의 사업에서 얻은 수입으로 한다.


제 2 장 회원과 회비(제5조～제7조)

제 5 조【회원】
  이 회의 회원은 주식회사에 재직하는 1달 이상 근무한 정직원으로 한다.
 
제 6 조【가입 및 탈퇴】
  ① 이 회의 가입은 전조에 정하는 회원의 자격을 얻은 때에 가입되며, 잃은 때에는 탈퇴된다. 
  ② 이 회의 가입은 종업원들의 자유의사로 결정하되 특별한 사유가 없는 한 전조에 정하는 회원의 자격을 가진 전 종업원이 동참함을 원칙으로 한다. 
  ③ 휴직 등의 사유로 회비를 납부할 수 없는 경우에는 자동 탈퇴한다. 단, 입영 휴직자의 경우 휴직기간 동안 상조회 회원의 자격을 정지한다.

제 7 조【회비】
1. 회비는 매월 급여 지급시 월 10,000원을 공제한다.
2. 근속년수 1년 이상시 100% 지급, 근속년수 1년 미만시 50% 지급한다.
3. 해당 회원의 지급 누적 한도액을 초과하여 지급은 불가하다.
다만, 회원 본인 결혼의 경우 지급 누적 한도를 초과하여 선지급이 가능하다.
4. 가입기간 동안 1회라도 지급받지 못한 회원이 퇴사하는 경우 연도별 미지급퇴사 한도에 따른 지급을 한다.
다만, 3년 이상 재직 회원의 선요구에 의해서만 지급을 한다.



Score: 0.8199362312577632
제 4 장 운  영(제11조～제19조)

제11조【운영위원회】
  운영위원회는 다음 사항을 의결한다.
    1. 규약의 제정 및 개정
    2. 중요 사

3. RAG 방식으로 LLM 결과 생성하기

In [9]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

Settings.llm = OpenAI(temperature=0.3, model="gpt-4o-mini")

folder_path = '../Assets/data/docs'
documents = SimpleDirectoryReader(folder_path).load_data()

## document array로 부터 벡터 스토어 생성하기
index = VectorStoreIndex.from_documents(
    documents,
)

query_engine = index.as_query_engine()
response = query_engine.query("사우회는 어떻게 운영되나요?")
print(response)

사우회는 운영위원회를 통해 운영되며, 이 위원회는 규약의 제정 및 개정, 중요 사업의 결정, 자금의 차입, 회의 해산, 감사의 선임 등 여러 중요한 사항을 의결합니다. 회장은 사무를 총괄하고 위원회의 의결사항을 집행하며, 감사는 회장을 보좌하고 기금을 관리합니다. 위원회는 과반수 이상의 찬성으로 의결하며, 정기회의와 임시회의를 통해 운영됩니다. 경조금 지급과 관련된 사항도 규정되어 있으며, 경조사유가 발생한 경우에는 지체 없이 신고해야 하며, 청구권은 발생일로부터 1개월 이내에 신청해야 합니다.


In [8]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=10)
Settings.text_splitter = text_splitter

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

response_synthesizer = get_response_synthesizer()

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

response = query_engine.query("사우회는 어떻게 운영되나요?")
print(response)

# 데이터 분석 결과

## 1. 커밋 빈도 및 패턴
- **일별 커밋 수**: 
  - 2024-07-22: 6
  - 2024-07-23: 12
  - 2024-07-24: 8
  - 2024-07-26: 2
- **주별 커밋 수**: 
  - 2024-07-21 ~ 2024-07-27: 총 28 커밋
- **요일별 커밋 패턴**: 
  - 주중에 비해 주말에 커밋이 적음.
- **시간대별 커밋 패턴**: 
  - 오전 9시 ~ 11시 사이에 가장 많은 커밋 발생.

## 2. 기여자 분석
- **가장 활발한 기여자**: 
  - Rain: 10 커밋
  - Hung: 9 커밋
  - Viet: 1 커밋
  - nhannt: 2 커밋
- **각 기여자의 기여도 분포**: 
  - Rain: 36%
  - Hung: 32%
  - Viet: 4%
  - nhannt: 7%

## 3. 커밋 메시지 분석
- **주요 키워드나 주제**: 
  - 버그 수정, 기능 추가, 리팩토링
- **작업 유형 분류**: 
  - 버그 수정: 17
  - 기능 추가: 7
  - 리팩토링: 0
- **특정 이슈나 프로젝트와 관련된 커밋 추적**: 
  - ncrm-kit.git 및 works-bprocess-svc.git에서의 활동이 두드러짐.

## 4. 파일 및 디렉토리 변경 분석
- **가장 자주 수정되는 파일**: 
  - pkg/reactor/event.go
  - pkg/reactor/handler/notification.go
- **코드베이스의 성장 추세**: 
  - 기능 추가와 버그 수정을 통해 코드베이스가 지속적으로 성장하고 있음.
- **특정 파일이나 디렉토리의 변경 이력**: 
  - notification.go 파일에서 여러 번의 수정이 이루어짐.

## 5. 코드 변경 규모
- **커밋별 추가/삭제된 라인 수**: 
  - 대부분의 커밋에서 추가 및 삭제된 라인 수가 적음 (+0, -0).
- **대규모 변경사항의 식별**: 
  - 몇몇 커밋에서 +M